## Connect to MySQL DB

In [69]:
pw = ''
host = ''
port = ''
db = ''

In [1]:
import mysql.connector
import pandas as pd

In [2]:
cnt = mysql.connector.connect(host=host, user='root', passwd=pw, database=db)

In [3]:
cursor = cnt.cursor(buffered=True)

In [4]:
# cursor.execute("SELECT * FROM telegram")
# result = cursor.fetchall()

In [7]:
# query = "DROP TABLE twitter"
# cursor.execute(query)

In [6]:
query = """CREATE TABLE twitter (
    id BIGINT(20) NOT NULL AUTO_INCREMENT PRIMARY KEY,
    created DATETIME NOT NULL,
    replies INT,
    retweets INT,
    likes INT,
    hashtag VARCHAR(255) NOT NULL)"""
cursor.execute(query)

## Add crawled data

In [70]:
pw = ''
host = ''
port = ''
db = ''

In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [57]:
def redundancy_check(sqlEngine, loaded_df):
    dbConnection = sqlEngine.connect()
    fetched_df = pd.read_sql("select * from crawling.twitter", dbConnection)
    # cleaned_df = loaded_df.merge(fetched_df, indicator=True, how='outer').loc[lambda x : x['_merge'] == 'left_only'].iloc[:, :-1]
    
    cleaned_df = loaded_df[~loaded_df.id.isin(fetched_df.id)]
    return cleaned_df
    

In [64]:
def update_db(tag='moonbirds', pw=None, host=None, port=None, db=None):
    df = pd.read_csv('crawled_{}.csv'.format(tag))

    table_name = 'twitter'
    db_url = 'mysql+pymysql://root:{}@{}:{}/{}'.format(pw, host, port, db)
    sqlEngine = create_engine(db_url, pool_recycle=3600)
    df = redundancy_check(sqlEngine, df)

    dbConnection = sqlEngine.connect()

    try:
        frame = df.to_sql(table_name, dbConnection, if_exists='append', chunksize=1000, index=False)
    except ValueError as vx:
        print(vx)
    except Exception as ex:   
        print(ex)
    else:
        print("Table %s created successfully." % table_name);   
    finally:
        dbConnection.close()

In [67]:
update_db(tag='moonbirds', 
          pw=pw, 
          host=host, 
          port=port, 
          db=db)

Table twitter created successfully.


In [68]:
update_db(tag='bayc', 
          pw=pw, 
          host=host, 
          port=port, 
          db=db)

Table twitter created successfully.
